The Transformer Quantum Inspired Deep Neural Network

Setting working dir

In [13]:
import os

In [14]:
os.chdir("C:/Users/Ábel/Desktop/Prog_nyelvek/ProjectWork/Algorithms/")

Instannilng up to date tensorflow and qiskit

In [15]:
!pip3 install tensorflow

In [16]:
!pip3 install qiskit

In [17]:
from qiskit import IBMQ

Load account and authenticate with IBM Quantum API token

In [18]:
if IBMQ.active_account() is None:
    IBMQ.save_account("689669e7018062b820af7391f76e14fee163cd2f7491b155fd9efc527f428bcfc47d126e838fb6aa25a035d3bd96137dd5d0120e53115fb13be89255e39fc9ff", overwrite=True)
    IBMQ.load_account()

C:\Users\Ábel\AppData\Local\Temp\ipykernel_17712\2240460110.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  if IBMQ.active_account() is None:
C:\Users\Ábel\AppData\Local\Temp\ipykernel_17712\2240460110.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  if IBMQ.active_account() is None:


Importing required packages

In [19]:
import pandas as pd
import yfinance as yf

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
from datetime import datetime, timedelta

Reading setup parametes from a text file

In [22]:
with open('parameters.txt', 'r') as file:
    lines = file.readlines()

In [23]:
start_date = lines[0].strip()
start_date = datetime.strptime(start_date, '%Y-%m-%d').replace(hour=0, minute=0, second=0)
end_date = lines[1].strip()
end_date = datetime.strptime(end_date, '%Y-%m-%d').replace(hour=0, minute=0, second=0)
keyword = lines[2].strip()
symbol = lines[3].strip()

In [24]:
start_date_format = start_date.strftime("%Y-%m-%d")
end_date_format = end_date.strftime("%Y-%m-%d")

In [25]:
file_path = f'Data/AverageSent/average_sentiments_{keyword}_{start_date_format}_{end_date_format}.csv'

Reading the sentiment csv file into a datafram

In [26]:
df_sentiment = pd.read_csv(file_path, parse_dates=['date'])

Downloading the stock data using Yahoo Finance package

In [27]:
df_stock = yf.download(symbol, start=start_date, end=end_date).reset_index()

[*********************100%***********************]  1 of 1 completed


Column names for both DataFrames for cheking, if the read and download was correct

In [28]:
print("Sentiment DataFrame columns:", df_sentiment.columns)
print("Stock DataFrame columns:", df_stock.columns)

Sentiment DataFrame columns: Index(['date', 'title_sentiment', 'content_sentiment',
       'description_sentiment'],
      dtype='object')
Stock DataFrame columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


Merge sentiment and stock price data based on the 'date' column

In [29]:
df = pd.merge(df_sentiment, df_stock, left_on='date', right_on='Date', how='inner')

In [30]:
print("Merged DataFrame columns:", df.columns)

Merged DataFrame columns: Index(['date', 'title_sentiment', 'content_sentiment', 'description_sentiment',
       'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
      dtype='object')


Drop unnecessary columns

In [31]:
df = df.dropna()

Convert the first column ('date') to datetime format

In [32]:
df['date'] = pd.to_datetime(df['date'])

Normalize numerical columns for consistent scaling

In [33]:
numerical_cols = ['title_sentiment', 'content_sentiment', 'description_sentiment', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

Min-max normalization for numerical columns

In [34]:
df[numerical_cols] = (df[numerical_cols] - df[numerical_cols].min()) / (df[numerical_cols].max() - df[numerical_cols].min())

Create input and output data

I will use the title, content, description sentiment and the Open price of a stock to predixt de closing price of a stock

In [35]:
X = df[['title_sentiment', 'content_sentiment', 'description_sentiment', 'Open']]
y = df['Close']

In [36]:
df = df.drop('Date', axis=1) 

Split into train/test

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train, 30% test

Split test data into test/validate (50% each for validation)

In [38]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)  # 50% test, 50% validation

In [39]:
import qiskit
from qiskit import QuantumCircuit, Aer, execute
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

Creating a simple quantum circuit using Qiskit

In [40]:
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

Function to simulate the quantum circuit

In [41]:
def simulate_quantum_circuit(circuit):
    aer_sim = Aer.get_backend("aer_simulator")
    job = execute(circuit, aer_sim, shots=1000)
    result = job.result()
    counts = result.get_counts()

    # Calculate the probability of measuring '0'
    prob_0 = counts.get("0", 0) / sum(counts.values())

    # Return as a TensorFlow constant with the (1,1)
    return tf.constant(prob_0, shape=(1, 1))


Keras layer for quantum-inspired logic

In [42]:
class QuantumInspiredLayer(keras.layers.Layer):
    def __init__(self, circuit):
        super().__init__()
        self.circuit = circuit

    def call(self, inputs):
        quantum_output = simulate_quantum_circuit(self.circuit)
        return inputs * quantum_output

Creating the model it self

Layers: 
    <br>
    1. Dense 1x64
    <br>
    2. Dense 1x128
    <br>
    3. Dense 1x256
    <br>
    4. Dense 1x512
    <br>
    5. Dense 1x1024
    <br>
    6. Dense 1x2048
    <br>
    7. quantum 
    <br>
    8. output- Dense 1x1
    <br>
    In case of every "normal" layer I used a dropout layer with 0.2 value for regularization
    and relu activator function

In [43]:
def create_model(input_shape):
    inputs = keras.Input(shape=(input_shape,))

    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.2)(x)  # Dropout for regularization
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    quantum_layer = QuantumInspiredLayer(qc)(x)
    
    output = layers.Dense(1)(quantum_layer)
    
    return keras.Model(inputs, output)

In [44]:
model = create_model(4)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

Early stopping

In [45]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]

In [46]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=100, 
    batch_size=32,
    callbacks=callbacks,
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.1671 - mae: 0.3453 - val_loss: 0.2432 - val_mae: 0.3146
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0850 - mae: 0.2471 - val_loss: 0.1620 - val_mae: 0.2684
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1147 - mae: 0.2311 - val_loss: 0.1552 - val_mae: 0.2684
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0991 - mae: 0.2146 - val_loss: 0.1722 - val_mae: 0.2644
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0573 - mae: 0.1967 - val_loss: 0.1868 - val_mae: 0.2648
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0561 - mae: 0.1948 - val_loss: 0.1813 - val_mae: 0.2639
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0620 - mae: 0.2071 - val_loss: 0.1655 - val_mae: 0.2665
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0447 - mae: 0.1817 - val_loss: 0.1358 - val_mae: 0.2701
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0469 - mae: 0.

Evaluate the model with the test data

In [47]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0221 - mae: 0.0992
Test loss: 0.022060321643948555, Test MAE: 0.09917312115430832


Evaluating the model with maching metrics for regression

In [48]:
train_loss, train_mae = model.evaluate(X_train, y_train)
val_loss, val_mae = model.evaluate(X_val, y_val)
test_loss, test_mae = model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0103 - mae: 0.0845
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0319 - mae: 0.1436
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0221 - mae: 0.0992


Train MAE

In [49]:
print("Train MAE:", train_mae)

Train MAE: 0.08449678868055344


This indicates that, on average, the predicted closing prices are about x units away from the actual closing prices in the training set.

Validation MAE

In [50]:
print("Validation MAE:", val_mae)

Validation MAE: 0.14362041652202606


A slightly higher MAE than the training set suggests some degree of overfitting, but the difference isn't dramatic. It could indicate that the model is generalizing fairly well.

Test MAE

In [51]:
print("Test MAE:", test_mae)

Test MAE: 0.09917312115430832


This is quite close to the train MAE, suggesting the model performs similarly on unseen data, which is a good sign.

In [52]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2048)           │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ quantum_inspired_layer          │ (None, 2048)           │             0 │
│ (QuantumInspiredLayer)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,399,429 (32.04 MB)

 Trainable params: 2,799,809 (10.68 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,599,620 (21.36 MB)